In [3]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchtext==0.10.0 -f https://download.pytorch.org/whl/cu111/torch_stable.html

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl (149.9 MB)
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     |█████████████                   | 834.1 MB 1.4 MB/s eta 0:14:34tcmalloc: large alloc 1147494400 bytes == 0x56072b5a8000 @  0x7f156c96f615 0x56072981f3bc 0x56072990018a 0x5607298221cd 0x560729914b3d 0x560729896458 0x56072989102f 0x560729823aba 0x5607298962c0 0x56072989102f 0x560729823aba 0x560729892cd4 0x560729915986 0x560729892350 0x560729915986 0x560729892350 0x560729915986 0x560729892350 0x560729823f19 0x560729867a79 0x560729822b32 0x5607298961dd 0x56072989102f 0x560729823aba 0x560729892cd4 0x56072989102f 0x560729823aba 0x560729891eae 0x5607298239da 0x560729892108 0x56072989102f
     |████████████████▌               | 1055.7 MB 1.5 MB/s eta 0:10:52tcmalloc: large alloc 1434370048 bytes == 0x56076fbfe000 @  0x7f156c96f615 0x56072981f3bc 0x56072990018a 0x5607298221cd 0x560729914b3d 0x560729

In [1]:
!pip install torch torchvision torchaudio pytorch-lightning

     |████████████████████████████████| 527 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 37.7 MB/s 
     |████████████████████████████████| 952 kB 30.9 MB/s 
     |████████████████████████████████| 134 kB 42.7 MB/s 
     |████████████████████████████████| 397 kB 44.4 MB/s 
     |████████████████████████████████| 829 kB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 29.1 MB/s 
     |████████████████████████████████| 271 kB 45.7 MB/s 
     |████████████████████████████████| 144 kB 40.9 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=be137d6b8ba490b861f554af7dd74e570ed5c2d83c1f3759e8f74197823c4c4c
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successful

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms as T
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

## Step 1: Define LightningModule

Under the hood, a LightningModule is still just a torch.nn.Module that groups all research code into a single file to make it self-contained:

 - The Train loop

 - The Validation loop

 - The Test loop

 - The Prediction loop

 - The Model or system of Models

 - The Optimizers and LR Schedulers

In [3]:
class LitAutoEncoder(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.encoder = nn.Sequential(
        nn.Linear(28 * 28, 64),
        nn.ReLU(),
        nn.Linear(64, 3))
    self.decoder = nn.Sequential(
        nn.Linear(3, 64),
        nn.ReLU(),
        nn.Linear(64, 28 * 28))
  

  def forward(self, x):
    # In lightning, forward defines the prediction/inference sections
    embedding = self.encoder(x)
    return embedding
  

  def training_step(self, batch, batch_idx):
    # Defines the train loop. Independent of forward
    x, y = batch
    x = x.view(x.size(0), -1)
    z = self.encoder(x)
    x_hat = self.decoder(z)
    loss = F.mse_loss(x_hat, x)
    # Logging to tensorboard by default
    self.log("train_loss", loss)
    return loss
  

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

You can customize any part of training (such as the backward pass) by overriding any of the 20+ hooks found in 
[Hooks](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html#lightning-hooks)

## Step 2: Fit with Lightning Trainer

In [4]:
dataset = MNIST(os.getcwd(), download=True, transform=T.ToTensor())
train_loader = DataLoader(dataset)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
# initilaize the model
autoencoder = LitAutoEncoder()
# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer(tpu_cores=8,
                     max_epochs=10,
                     limit_train_batches=10) # skipping batches for faster training
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: True, using: 8 TPU cores
IPU available: False, using: 0 IPUs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/plugins/training_type/tpu_spawn.py:204: UserWarning: cleaning up tpu spawn environment...
  rank_zero_warn("cleaning up tpu spawn environment...")
